# 外部moduleと関数の読み込み

In [1]:
import duckdb
import io
import zipfile
import urllib.request
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
con = duckdb.connect('/Users/nagampere/File/horkew/projects/tokyo2018/dbt/catalog.duckdb')

In [2]:
def read_geojson(url:str)->gpd.GeoDataFrame():
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as res:
        data = res.read()
    # データがzipファイルになっているので中身を抽出して変数に格納する
    with zipfile.ZipFile(io.BytesIO(data), 'r') as zip_data:
        filepath = [name for name in zip_data.namelist() if '.geojson' in name][0]
        
        with zip_data.open(filepath, 'r') as data:
            shp = gpd.read_file(data)
    return shp

# 医療施設データとODデータの読み込み

In [3]:
# 病院の地理データの読み込み
gdf_ibaraki = read_geojson('https://nlftp.mlit.go.jp/ksj/gml/data/P04/P04-20/P04-20_08_GML.zip')
gdf_saitama = read_geojson('https://nlftp.mlit.go.jp/ksj/gml/data/P04/P04-20/P04-20_11_GML.zip')
gdf_chiba = read_geojson('https://nlftp.mlit.go.jp/ksj/gml/data/P04/P04-20/P04-20_12_GML.zip')
gdf_tokyo = read_geojson('https://nlftp.mlit.go.jp/ksj/gml/data/P04/P04-20/P04-20_13_GML.zip')
gdf_kanagawa = read_geojson('https://nlftp.mlit.go.jp/ksj/gml/data/P04/P04-20/P04-20_14_GML.zip')
gdf_hospital = pd.concat([gdf_ibaraki,gdf_saitama,gdf_chiba,gdf_tokyo,gdf_kanagawa],ignore_index=True)
gdf_hospital = gdf_hospital.rename(
    columns={
        "P04_001":"医療機関分類",
        "P04_002":"施設名称",
        "P04_003":"所在地",
        "P04_004":"診療科目１",
        "P04_005":"診療科目２",
        "P04_006":"診療科目３",
        "P04_007":"開設者分類",
        "P04_008":"病床数",
        "P04_009":"救急告示病院",
        "P04_010":"災害拠点病院"
    }
)
gdf_hospital["医療機関分類"] = gdf_hospital["医療機関分類"].replace({1:'病院',2:'診療所',3:'歯科診療所'})
gdf_hospital["開設者分類"] = gdf_hospital["開設者分類"].replace({1:'国',2:'公的医療機関',3:'社会保険関係団体',4:'医療法人',5:'個人',6:'その他',9:'分析対象外'})
gdf_hospital["救急告示病院"] = gdf_hospital["救急告示病院"].replace({1:'指定あり',9:'指定なし'})
gdf_hospital["災害拠点病院"] = gdf_hospital["災害拠点病院"].replace({1:'基幹',2:'地域',9:'指定なし'})
gdf_hospital["medical_facility_lat"] = gdf_hospital["geometry"].y
gdf_hospital["medical_facility_lon"] = gdf_hospital["geometry"].x


In [4]:
# ODデータの読み込み
df_ods = con.sql(
    """
    select
        *
    from int_ods
    """
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
gdf_ods_origin = gpd.GeoDataFrame(df_ods,
                                  geometry=gpd.points_from_xy(df_ods['origin_lon'],df_ods['origin_lat'],crs='EPSG:6668'))
gdf_ods_destination = gpd.GeoDataFrame(df_ods,
                                  geometry=gpd.points_from_xy(df_ods['destination_lon'],df_ods['destination_lat'],crs='EPSG:6668'))

# 空間結合

In [6]:
#CRSの変換
gdf_hospital = gdf_hospital.to_crs(2451)
gdf_ods_origin = gdf_ods_origin.to_crs(2451)
gdf_ods_destination = gdf_ods_destination.to_crs(2451)

In [7]:
gdf_ods_origin = gpd.sjoin_nearest(gdf_ods_origin,
                                   gdf_hospital,
                                   how='left',
                                   max_distance=100)

In [8]:
gdf_ods_destination = gpd.sjoin_nearest(gdf_ods_destination,
                                        gdf_hospital,
                                        how='left',
                                        max_distance=100)

# 通院目的の絞り込み

In [9]:
gdf_ods_origin_hospital = gdf_ods_origin.query('purpose_origin == "07通院・リハビリ"')
gdf_ods_destination_hospital = gdf_ods_destination.query('purpose_destination == "07通院・リハビリ"')

In [24]:
gdf_ods_origin_hospital.to_csv('output/ods_from_hospital.csv', encoding='CP932',index=False)
gdf_ods_destination_hospital.to_csv('output/ods_to_hospital.csv', encoding='CP932',index=False)
gdf_hospital.to_csv('output/hospital.csv',encoding='CP932',index=False)
gdf_ods_origin_hospital.to_csv('output/ods_from_hospital_UTF8.csv', index=False)
gdf_ods_destination_hospital.to_csv('output/ods_to_hospital_UTF8.csv', index=False)
gdf_hospital.to_csv('output/hospital_UTF8.csv', index=False)

# 医療施設とODデータの可視化

In [15]:
# load the config
%run config_origin.py

# set the config to kepler.gl
map_origin = KeplerGl(
    height=500,
    data={"medical_facility": gdf_hospital, "ods": gdf_ods_origin_hospital},
    config=config_origin
)
map_origin

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/nagampere/File/horkew/.venv/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['medical_facility'], 'id': '…

In [25]:
# save the config
with open('config_origin.py', 'w') as f:
   f.write('config_origin = {}'.format(map_origin.config))

In [26]:
# save the map
map_origin.save_to_html(file_name='output/map_origin.html')

Map saved to output/map_origin.html!


In [19]:

map_destination = KeplerGl(
    height=500,
    data={"medical_facility": gdf_hospital, "ods": gdf_ods_destination_hospital}
    # config=config_destination
)
map_destination

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/nagampere/File/horkew/.venv/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'medical_facility':       医療機関分類                 施設名称                所在地            診療科目１ 診療科目２…

In [27]:
# save the config
# with open('config_destination.py', 'w') as f:
#    f.write('config_destination = {}'.format(map_destination.config))

In [28]:
# save the map
map_destination.save_to_html(file_name='output/map_destination.html')

Map saved to output/map_destination.html!
